In [2]:
import pandas as pd

# importing json into pandas dataframe structure
proj_df = pd.read_json('combined.json', lines=True)

# converting dataframe to numpy array
proj_array = proj_df.values

In [9]:
# double checking the shape of array
proj_array[0]
#proj_array.shape

array([list(['National Security Division (NSD)']),
       'PORTLAND, Oregon. – Mohamed Osman Mohamud, 23, who was convicted in 2013 of attempting to use a weapon of mass destruction (explosives) in connection with a plot to detonate a vehicle bomb at an annual Christmas tree lighting ceremony in Portland, was sentenced today to serve 30 years in prison, followed by a lifetime term of supervised release. Mohamud, a naturalized U.S. citizen from Somalia and former resident of Corvallis, Oregon, was arrested on Nov. 26, 2010, after he attempted to detonate what he believed to be an explosives-laden van that was parked near the tree lighting ceremony in Portland.\xa0 The arrest was the culmination of a long-term undercover operation, during which Mohamud was monitored closely for months as his bomb plot developed.\xa0 The device was in fact inert, and the public was never in danger from the device. At sentencing, United States District Court Judge Garr M. King, who presided over Mohamed’s 